In [1]:
library(tidyverse)
library(tidymodels)
library(bonsai)
tidymodels_prefer()

x_feats <- c('X_Minimum', 'X_Maximum','Y_Minimum', 'Y_Maximum', 'Pixels_Areas', 'X_Perimeter', 'Y_Perimeter', 'Sum_of_Luminosity', 'Minimum_of_Luminosity', 'Maximum_of_Luminosity', 'Length_of_Conveyer', 'TypeOfSteel_A300', 'TypeOfSteel_A400', 'Steel_Plate_Thickness', 'Edges_Index', 'Empty_Index', 'Square_Index', 'Outside_X_Index', 'Edges_X_Index', 'Edges_Y_Index', 'Outside_Global_Index', 'LogOfAreas', 'Log_X_Index', 'Log_Y_Index', 'Orientation_Index', 'Luminosity_Index', 'SigmoidOfAreas')
Y_feats <- c('Pastry', 'Z_Scratch', 'K_Scatch', 'Stains', 'Dirtiness', 'Bumps', 'Other_Faults')

train_df <- read_csv("/kaggle/input/playground-series-s4e3/train.csv") %>%
    pivot_longer(cols = Y_feats, names_to = "faults") %>%
    filter(value == 1) %>%
    select(-value) %>%
    mutate(faults = as.factor(faults))

test_df <- read_csv("/kaggle/input/playground-series-s4e3/test.csv")
glimpse(train_df)

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.4
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.4.4     ✔ tibble    3.2.1
✔ lubridate 1.9.3     ✔ tidyr     1.3.0
✔ purrr     1.0.2     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors
── Attaching packages ────────────────────────────────────── tidymodels 1.1.1 ──

✔ broom        1.0.5     ✔ rsample      1.2.0
✔ dials        1.2.0     ✔ tune         1.1.2
✔ infer        1.0.5     ✔ workflows    1.1.3
✔ modeldata    1.2.0     ✔ workflowsets 1.0.1
✔ parsnip      1.1.1     ✔ yardstick    1.2.0
✔ recipes      1.0.8     

── Conflicts ───────────────────────────────────────── tidymodels_conflicts() ──
✖ scales::discard() masks purrr::discard()
✖ dplyr::filt

Rows: 18,422
Columns: 29
$ id                    <dbl> 0, 1, 2, 3, 4, 5, 6, 7, 9, 10, 11, 13, 14, 15, 1…
$ X_Minimum             <dbl> 584, 808, 39, 781, 1540, 1009, 596, 1673, 893, 1…
$ X_Maximum             <dbl> 590, 816, 192, 789, 1560, 1033, 607, 1687, 907, …
$ Y_Minimum             <dbl> 909972, 728350, 2212076, 3353146, 618457, 899231…
$ Y_Maximum             <dbl> 909977, 728372, 2212144, 3353173, 618502, 899307…
$ Pixels_Areas          <dbl> 16, 433, 11388, 210, 521, 409, 204, 571, 54, 289…
$ X_Perimeter           <dbl> 8, 20, 705, 16, 72, 22, 21, 38, 6, 37, 27, 14, 3…
$ Y_Perimeter           <dbl> 5, 54, 420, 29, 67, 26, 21, 57, 15, 42, 29, 13, …
$ Sum_of_Luminosity     <dbl> 2274, 44478, 1311391, 3202, 48231, 47513, 22478,…
$ Minimum_of_Luminosity <dbl> 113, 70, 29, 114, 82, 86, 89, 77, 87, 84, 105, 1…
$ Maximum_of_Luminosity <dbl> 140, 111, 141, 134, 111, 118, 127, 110, 133, 127…
$ Length_of_Conveyer    <dbl> 1358, 1687, 1400, 1387, 1692, 1650, 1373, 1692, …
$ TypeOfSteel_A

In [2]:
tune_spec <-
    boost_tree(
#         trees = tune(),
#         tree_depth = tune(), 
#         learn_rate = tune(),
#         min_n = tune(),
        loss_reduction = tune()
    ) %>%
    set_engine("lightgbm") %>%
    set_mode("classification")

In [3]:
grid <- grid_regular(
#     trees(),
#     tree_depth(),
#     learn_rate(),
#     min_n(),
    loss_reduction(),
    levels = 3
)

In [4]:
set.seed(1234)
folds <- vfold_cv(train_df, v = 3)

In [5]:
wf <- workflow() %>%
    add_model(tune_spec) %>%
    add_formula(faults ~ . -id)

In [6]:
res <- wf %>% 
    tune_grid(
        resamples = folds,
        grid = grid
    )

In [7]:
best <- res %>%
  select_best("roc_auc")

best

loss_reduction,.config
<dbl>,<chr>
5.623413e-05,Preprocessor1_Model2


In [8]:
final_wf <- wf %>% 
  finalize_workflow(best)

In [9]:
final_fit <- 
  final_wf %>%
  fit(train_df)

In [10]:
p <- predict(final_fit, new_data = test_df, type = "prob")

In [11]:
names(p) <- str_replace(names(p), ".pred_", "")
p %>% glimpse

Rows: 12,814
Columns: 7
$ Bumps        <dbl> 0.1594106955, 0.1402409796, 0.3039801090, 0.4148920324, 0…
$ Dirtiness    <dbl> 0.0066266866, 0.3815004536, 0.0053476400, 0.0066008073, 0…
$ K_Scatch     <dbl> 0.006601115, 0.009488231, 0.051726131, 0.001156029, 0.001…
$ Other_Faults <dbl> 0.27186119, 0.21413990, 0.58094457, 0.44675154, 0.3558481…
$ Pastry       <dbl> 0.5543466967, 0.2469835466, 0.0019098005, 0.1290015051, 0…
$ Stains       <dbl> 1.655978e-04, 1.667007e-04, 5.134799e-04, 3.174355e-04, 7…
$ Z_Scratch    <dbl> 0.0009880220, 0.0074801861, 0.0555782659, 0.0012806471, 0…


In [12]:
p %>%
    mutate(id = test_df$id) %>%
    write_csv("submission.csv")